
# THE DEPENDENCIES

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from keras.models import Sequential,load_model
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras.layers import Flatten
from datetime import datetime
from keras.layers import Conv2D
from keras.layers import Dense
from claptcha import Claptcha
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os,os.path
import imutils
import random
import string
import glob
import cv2
import tensorflow
%matplotlib inline




# CREATING CAPTCHAS RANDOMLY

In [3]:
def randomString():
    rndLetters = (random.choice(string.ascii_uppercase+string.digits) for _ in range(4))
    return "".join(rndLetters)
def match(str1, str2): 
    c, j = 0, 0
    for i in str1:    
        if str2.find(i)>= 0 and j == str1.find(i): 
            c += 1
        j += 1
    if c == 3 :
        return 0.75
    elif c == 2 :
        return 0.5
    elif c == 4 :
        return 1
    else :
        return 0

In [4]:
captcha_text = []
n = 2000
for i in range(1,n):
    text = randomString()
    c = Claptcha(text, "FreeMono.ttf", (150,90),
             resample=Image.BICUBIC, noise=0.2)
    c.margin = (25,25)
    text, _ = c.write('Generated_captchas\\'+text+'.png')
    captcha_text.append(text)

C:\Users\hp\AppData\Local\Temp\ipykernel_1424\2882395662.py:6: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC, noise=0.2)


In [ ]:
a = pd.Series(captcha_text)
a.to_csv('Captcha_label.csv')

# PREPROCESSING THE GENERATED CAPTCHAS

In [ ]:
captchas_folder = "Generated_captchas/"
preprocessed_captchas = "Preprocessed/"

In [ ]:
def saveimage(letter_text,letter_image):
    path = os.path.join(preprocessed_captchas, letter_text)
    if not os.path.exists(path):
        os.makedirs(path)
    p = os.path.join(path, str(random.randint(1,10000))+".png")
    cv2.imwrite(p, letter_image)

In [ ]:
captcha_images=  glob.glob(os.path.join(captchas_folder, "*"))

# SAMPLE IMAGE PREPROCESSING AND DRAWING COUNTERS

In [ ]:
image = cv2.imread(captcha_images[3])
Image.fromarray(image)

In [ ]:
image.shape

In [ ]:
s = Image.open(captcha_images[1])

In [ ]:
s.getpixel((2,5))

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
Image.fromarray(gray)

In [ ]:
ret,thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
Image.fromarray(thresh)

In [ ]:



contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)


image_copy = gray.copy()

cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

                

cv2.imshow('None approximation', image_copy)

cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', image_copy)
cv2.destroyAllWindows()


In [ ]:
img = cv2.drawContours(image,contours,1,(255,0,0),3)
Image.fromarray(img)

# Exploring the Image 

In [ ]:
img = cv2.drawContours(image,contours,0,(255,255,0),3)
Image.fromarray(img)

In [ ]:
img = cv2.drawContours(image,contours,-1,(255,0,0),3)
Image.fromarray(img)

In [ ]:
for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
    print(x,y,w,h)

In [ ]:
Image.fromarray(thresh[0:thresh.shape[0],10:40])

# SAVING PREPROCESSED IMAGES

In [ ]:
for file in captcha_images:
    mystr = file.split('/')
    value = mystr[-1].split('\\')[1].split('.')[0]
    image = cv2.imread(file)
    gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    image = cv2.threshold(gray_scale,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
    slicedImage = image[0:image.shape[0],10:40]
    letter = value[0]
    saveimage(letter,slicedImage) 
    slicedImage = image[0:image.shape[0],40:70]
    letter = value[1]
    saveimage(letter,slicedImage)
    slicedImage = image[0:image.shape[0],70:100]
    letter = value[2]
    saveimage(letter,slicedImage)
    slicedImage = image[0:image.shape[0],100:130]
    letter = value[3]
    saveimage(letter,slicedImage)

# GETTING STARTED WITH TRAINING PHASE

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
training_set = train_datagen.flow_from_directory('Preprocessed/',
target_size = (32, 32),
class_mode = 'categorical',color_mode='grayscale')

testing_set = test_datagen.flow_from_directory('Test_data//',
target_size = (32, 32),
class_mode = 'categorical',color_mode='grayscale')


In [ ]:
training_set.classes

# THE NEURAL NETWORK ARCHITECTURE

In [ ]:
%reload_ext tensorboard
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520',write_graph=True)

In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, (5, 5), input_shape = (32, 32, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=(2,2)))
classifier.add(Conv2D(32, (5, 5), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units = 500, activation = 'relu'))
classifier.add(Dense(units = 36, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(training_set,steps_per_epoch=20,epochs=100, callbacks=[tboard_callback])

In [ ]:
%tensorboard --logdir=logs

In [ ]:
d = training_set.class_indices
print(d)


# PRESERVING THE TRAINED MODEL

In [ ]:
classifier.save('captchaclassifier.hdf5')

# LOADING MODEL AND DEPLOYMENT

In [ ]:
captcha_model = load_model('captchaclassifier.hdf5')

In [ ]:

captcha_image_files =  glob.glob(os.path.join('Test_data\\Test', "*"))

In [ ]:
dict_class = {v:k for k,v in d.items()}
print(captcha_image_files)
print("\n")
print(dict_class)



# MAPPING THE LABELS WITH VALUES

In [ ]:
def find_value(pred):
    count =0
    for i in pred[0] :
        count+=1
        if i >= 0.5:
            return dict_class[count-1]


# Prediction logic for the test data

In [ ]:
count=0.0
for file in captcha_image_files:
    combine = []
    mystr = file.split('\\')
    value1 = mystr[-1].split('.')[0]
    image = cv2.imread(file)
    gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.threshold(gray_scale,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
    slicedImage = image[0:90,10:40]
    resized = cv2.resize(slicedImage,(32,32))
    img= np.expand_dims(resized,axis=2)
    img = np.expand_dims(img,axis=0)
    
    pred = captcha_model.predict(img)
    
    print((pred[0]))
    
    
    first = find_value(pred)
    print(first)
    combine.append(first)
     
    ##
    slicedImage = image[0:90,40:70]
    resized = cv2.resize(slicedImage,(32,32))
    img= np.expand_dims(resized,axis=2)
    img = np.expand_dims(img,axis=0)
    
    
    pred = captcha_model.predict(img)
    
    print((pred[0]))
    
    
    second = find_value(pred)
    print(second)
    combine.append(second)

    ##
    slicedImage = image[0:90,70:100]
    resized = cv2.resize(slicedImage,(32,32))
    img= np.expand_dims(resized,axis=2)
    img = np.expand_dims(img,axis=0)
    
    pred = captcha_model.predict(img)
    
    print((pred[0]))
    
    
    third = find_value(pred)
    print(third)
    combine.append(third)
    print(combine)
    
    ##
    slicedImage = image[0:90,100:130]
    resized = cv2.resize(slicedImage,(32,32))
    img= np.expand_dims(resized,axis=2)
    img= np.expand_dims(img,axis=0)
    pred = captcha_model.predict(img)
    
    print((pred[0]))
    
    
    fourth = find_value(pred)
    print(fourth)
    combine.append(fourth)
    print(combine)
    
    predict = ''.join(combine)
    print(predict)
    print(value1)
    print("------------")
    result = match(value1,predict)
    if result == 0.75 :
        print("String Matched by 75%")
    elif result == 0.5 :
        print("String Matched by 50%")
    elif result == 1 :
        print("String Matched by 100%")
    else :
        print("String havent matched")
    count+=match(value1,predict)
    print(match(value1,predict))
    '''if value1 == predict:
        count+=1'''